# 04 - Normalização de Indicadores

Este notebook realiza:
1. Filtro de indicadores ativos (CONSIDERAR? = SIM)
2. Normalização de valores (0-100) **por grupo de posição + competição**
3. Tratamento de direção (CIMA vs BAIXO)

**Importante**: A normalização é feita separadamente para cada combinação de `mapped_position + competition_id`.
Exemplo: Goleiros (GK) da Serie A são normalizados apenas entre si, não se misturam com GKs de outras ligas nem com outras posições.

In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path

BASE_DIR = Path("c:/jobs/botafogo/v3")
OUTPUT_DIR = BASE_DIR / "bases" / "outputs"

## 1. Carregar Dados

In [ ]:
# Carregar dados consolidados
df = pd.read_parquet(OUTPUT_DIR / "_temp_scouts_consolidated.parquet")
print(f"Jogadores carregados: {len(df)}")

# Carregar pesos ativos
df_weights = pd.read_parquet(OUTPUT_DIR / "_temp_weights_active.parquet")
print(f"Indicadores ativos: {len(df_weights)}")

In [ ]:
# Verificar estrutura dos pesos
print("Colunas da tabela de pesos:")
print(df_weights.columns.tolist())

In [ ]:
# Verificar indicadores
print(f"\nIndicadores únicos: {df_weights['INDICADOR'].nunique()}")
print(f"\nDireção (Melhor para):")
print(df_weights["Melhor para"].value_counts())

## 2. Identificar Indicadores Válidos

In [ ]:
# Lista de indicadores que devem ser normalizados
indicadores = df_weights["INDICADOR"].str.strip().tolist()

# Verificar quais existem no DataFrame de scouts
indicadores_disponiveis = [ind for ind in indicadores if ind in df.columns]
indicadores_faltantes = [ind for ind in indicadores if ind not in df.columns]

print(f"Indicadores na tabela de pesos: {len(indicadores)}")
print(f"Indicadores disponíveis nos scouts: {len(indicadores_disponiveis)}")
print(f"Indicadores faltantes: {len(indicadores_faltantes)}")

if indicadores_faltantes:
    print(f"\nPrimeiros 10 indicadores faltantes:")
    for ind in indicadores_faltantes[:10]:
        print(f"  - {ind}")

In [ ]:
# Criar mapeamento de direção para cada indicador
direction_map = dict(zip(
    df_weights["INDICADOR"].str.strip(),
    df_weights["Melhor para"]
))

print(f"Mapeamento de direção criado para {len(direction_map)} indicadores")

## 3. Função de Normalização

In [ ]:
def normalize_column(series, direction="CIMA"):
    """
    Normaliza uma série de valores para o intervalo 0-100.
    
    Args:
        series: pd.Series com os valores a normalizar
        direction: 'CIMA' (maior é melhor) ou 'BAIXO' (menor é melhor)
    
    Returns:
        pd.Series normalizada (0-100)
    """
    # Ignorar NaN para cálculos
    min_val = series.min()
    max_val = series.max()
    
    # Evitar divisão por zero
    if max_val == min_val:
        return pd.Series([50.0] * len(series), index=series.index)
    
    # Normalização
    if direction == "CIMA":
        # Maior valor = 100
        normalized = (series - min_val) / (max_val - min_val) * 100
    else:  # BAIXO
        # Menor valor = 100
        normalized = (max_val - series) / (max_val - min_val) * 100
    
    return normalized

## 4. Aplicar Normalização

In [ ]:
# Criar DataFrame para valores normalizados
df_normalized = df.copy()

# Criar coluna de grupo para normalização: posição + competição
df_normalized["_norm_group"] = df_normalized["mapped_position"].astype(str) + "_" + df_normalized["competition_id"].astype(str)

# Verificar grupos criados
grupos = df_normalized["_norm_group"].nunique()
print(f"Grupos de normalização (posição + competição): {grupos}")
print(f"\nExemplos de grupos:")
print(df_normalized["_norm_group"].value_counts().head(10))

# Normalizar cada indicador por grupo
normalized_count = 0
errors = []

for indicador in indicadores_disponiveis:
    try:
        direction = direction_map.get(indicador, "CIMA")
        
        # Converter para numérico se necessário
        df_normalized[indicador] = pd.to_numeric(df_normalized[indicador], errors="coerce")
        
        # Normalizar POR GRUPO (posição + competição)
        df_normalized[f"{indicador}_norm"] = df_normalized.groupby("_norm_group")[indicador].transform(
            lambda x: normalize_column(x, direction)
        )
        normalized_count += 1
        
    except Exception as e:
        errors.append((indicador, str(e)))

# Remover coluna auxiliar
df_normalized.drop(columns=["_norm_group"], inplace=True)

print(f"\nIndicadores normalizados: {normalized_count}")
if errors:
    print(f"\nErros ({len(errors)}):")
    for ind, err in errors[:5]:
        print(f"  - {ind}: {err}")

In [ ]:
# Verificar normalização
norm_cols = [c for c in df_normalized.columns if c.endswith("_norm")]
print(f"\nColunas normalizadas criadas: {len(norm_cols)}")

# Estatísticas de uma coluna normalizada
if norm_cols:
    sample_col = norm_cols[0]
    print(f"\nEstatísticas de '{sample_col}':")
    print(df_normalized[sample_col].describe())

## 5. Verificar Valores Normalizados

In [ ]:
# Verificar se todos os valores estão no range 0-100
out_of_range = 0
for col in norm_cols:
    values = df_normalized[col].dropna()
    if (values < 0).any() or (values > 100).any():
        out_of_range += 1
        print(f"  {col}: min={values.min():.2f}, max={values.max():.2f}")

if out_of_range == 0:
    print("Todos os valores normalizados estão no range 0-100!")
else:
    print(f"\n{out_of_range} colunas com valores fora do range")

In [ ]:
# Exemplo de normalização (antes e depois)
if indicadores_disponiveis:
    sample_ind = indicadores_disponiveis[0]
    print(f"\nExemplo: {sample_ind}")
    print(f"Direção: {direction_map.get(sample_ind, 'CIMA')}")
    print(f"\nAntes (original):")
    print(df_normalized[sample_ind].describe())
    print(f"\nDepois (normalizado):")
    print(df_normalized[f"{sample_ind}_norm"].describe())

## 6. Criar Mapeamento de Indicadores para Pesos

In [ ]:
# Criar DataFrame com mapeamento indicador -> peso por posição
position_columns = ["GK", "RCB", "LCB", "CB", "RB", "LB", "DM", "CM", "AM", "LW", "RW", "CF"]

# Verificar quais colunas de posição existem
available_pos_cols = [c for c in position_columns if c in df_weights.columns]
print(f"Colunas de posição disponíveis: {available_pos_cols}")

# Criar mapeamento indicador -> pesos
weights_dict = {}
for _, row in df_weights.iterrows():
    indicador = row["INDICADOR"].strip()
    if indicador in indicadores_disponiveis:
        weights_dict[indicador] = {}
        for pos in available_pos_cols:
            weights_dict[indicador][pos] = row[pos] if pd.notna(row[pos]) else 0

print(f"\nMapeamento de pesos criado para {len(weights_dict)} indicadores")

## 7. Salvar Dados Normalizados

In [ ]:
# Salvar DataFrame normalizado
df_normalized.to_parquet(OUTPUT_DIR / "_temp_scouts_normalized.parquet", index=False)
print(f"Dados normalizados salvos: {OUTPUT_DIR / '_temp_scouts_normalized.parquet'}")

# Salvar mapeamento de pesos como JSON para próxima etapa
import json
with open(OUTPUT_DIR / "_temp_weights_map.json", "w") as f:
    json.dump(weights_dict, f)
print(f"Mapeamento de pesos salvo: {OUTPUT_DIR / '_temp_weights_map.json'}")

# Salvar lista de indicadores disponíveis
with open(OUTPUT_DIR / "_temp_indicators_available.json", "w") as f:
    json.dump(indicadores_disponiveis, f)
print(f"Lista de indicadores salva: {OUTPUT_DIR / '_temp_indicators_available.json'}")

In [ ]:
# Resumo
print("=" * 50)
print("RESUMO DA NORMALIZAÇÃO")
print("=" * 50)
print(f"Jogadores: {len(df_normalized)}")
print(f"Indicadores normalizados: {len(norm_cols)}")
print(f"Colunas totais: {len(df_normalized.columns)}")
print("=" * 50)